In [2]:
from astropy.constants import G, c, M_sun, R_sun
from math import pi
from astropy import units as u
from math import pow
import numpy as np

# For LMC X-1 (M1 = Black Hole, M2 = Companion).

Orosz, J. A., Steeghs, D., McClintock, J. E., Torres, M. A. P., Bochkov, I., Gou, L., et al. (2009). A NEW DYNAMICAL MODEL FOR THE BLACK HOLE BINARY LMC X-1. The Astrophysical Journal, 697(1), 573–591. http://doi.org/10.1088/0004-637X/697/1/573

Also 

Hyde 2017: https://ui.adsabs.harvard.edu/abs/2017PASP..129i4201H/abstract

...especially Table 4

In [3]:
m1 = 10.8 * u.Msun
m2 = 31.79 * u.Msun
period = 3.909 * u.day
r2 = 17.0 * u.Rsun
vwind_inf = 1400 * u.km / u.s
inclin = 36.38 * u.deg
r2_rot = ((130) * u.km / u.s) / np.sin(inclin)

# Orbital Parameters

In [4]:
period = period.to(u.yr)
a = (((m1 + m2) * period**2)**(1/3)).value * u.au
print(f'Separation: {a:8.4}')
print(f'Separatiom (light seconds): {a.to(u.m)/c:8.4}')
print(f'Separation : {a.to(u.Rsun)}')


# BH orbital velocity:
orb_vel = np.sqrt(G * (m1 + m2).to(u.kg) / a.to(u.m))
orb_vel = orb_vel.to(u.km / u.s)
print(f"Orbital velocity: {orb_vel:6.5}")
print()

Separation:   0.1696 AU
Separatiom (light seconds):    84.63 s
Separation : 36.46892143322825 solRad
Orbital velocity: 471.99 km / s



In [8]:
# Non-spinning black hole r_G
rG = (2 * m1.to(u.kg) * G) / (c**2)
print(f'Schwarzschild Radius: {rG:10.7}')
print(f'Schwarzschild Radius: {(rG/c).to(u.us):10.7}')
print(f'Schwarzschild Radius: {rG.to(u.earthRad):8.3}')
print(f'Schwarzschild Radius: {rG.to(u.Rsun):8.3}')




Schwarzschild Radius:    31895.1 m
Schwarzschild Radius:   106.3906 us
Schwarzschild Radius:    0.005 earthRad
Schwarzschild Radius: 4.58e-05 solRad


# Wind accretion calculations

Shakura 2014: https://arxiv.org/pdf/1407.3163.pdf

Negueruela http://adsabs.harvard.edu/full/2010ASPC..422...57N

Edgar 2004: https://ui.adsabs.harvard.edu/abs/2004NewAR..48..843E/abstract

Disk physics: https://www-aanda-org.clsproxy.library.caltech.edu/articles/aa/pdf/2019/02/aa34498-18.pdf


In [14]:
from astropy import units as u

def wind_velocity(r, beta = 1.0, v_inf = 1000*u.km / u.s):
    '''
    From Negueruela eqn 1. Computes the wind velocity for a line driven
    wind as a function of distance from the star.
    
    Parameters
    ----------
    r: float
        Distane, scaled by the radius of the OB star (i.e. r = r_d / R_star)

    '''
    
    return v_inf * (1 - r**-1)**(beta)



In [15]:
# Tangential rotational velocity at BH:
v_tan = orb_vel -  (r2 / a.to(u.Rsun))* r2_rot

wind_speed = wind_velocity(a.to(u.m) / r2.to(u.m), v_inf =vwind_inf )
print(f'Wind speec at BH: {wind_speed}')
v_rel = np.sqrt(wind_speed**2 + v_tan**2)
print(f'Relative local speed of gas to BH: {v_rel:8.4}')

r_acc = (2 * G * m1.to(u.kg) / (v_rel**2)).to(u.Rsun)
print(f'Accretion radius: {r_acc:9.4}')
print(f'Accretion radius / separation: {r_acc.to(u.au)/a:9.4}')
print(f'Accretion radius / rG :{r_acc / rG.to(u.Rsun):9.4}')




Wind speec at BH: 747.389528819054 km / s
Relative local speed of gas to BH:    833.9 km / s
Accretion radius:     5.926 solRad
Accretion radius / separation:    0.1625
Accretion radius / rG :1.292e+05


In [106]:
# M =M =4.52 ́10-7M yr from Vink et al 2001:
m2_massloss = 4.52e-7 * u.Msun / u.yr
density = (m2_massloss / (4 * np.pi * a**2 * wind_speed)).to(u.g/u.cm**3)
print(f'Density of wind at BH: {density:8.4}')

# Bondi-Hoyle 1944:

m1_mass_rate = (np.pi * r_acc**2 * density * v_rel).to(u.Msun / u.yr)
print(f'Accretion rate: {m1_mass_rate:9.4}')

In [115]:
# Free-fall timescale:
tau_ff = np.sqrt(3 * np.pi / (G * density)).to(u.s)
print(tau_ff)
print(density)

In [115]:
# Angular momentum of material in the accretion capture:




173135398918.53555 s
4.7109443658290194e-15 g / cm3


# Roche Lobe of the Secondary

Eggelton ratio
http://articles.adsabs.harvard.edu/pdf/1983ApJ...268..368E

In [9]:
q = m2 / m1
eggleton_ratio = (0.49 * q ** (2/3.)) / (0.6 * q**(2/3.) + np.log(1 + q**(1/3.)) )
print(f'Eggeleton Ratio: {eggleton_ratio:5.3}')

Eggeleton Ratio: 0.474


In [10]:
q2 = m1/ m2
#vel_bh = 200.0 * ( (m1 / M_sun)**(1/3.) )*((1.0 + q2)**(1/3.))*period.value**(-1/3.) * u.km / u.s
#print(f"Approximate orbital velocity: {vel_bh}")
#print()
vel = np.sqrt(G * (m1 + m2) / a)
print(f"Orbital velocity: {vel}")
print()


frac_captured = (0.25 ) * (m1 / m2)**2 * (r2 / a)
print(f"Fraction of wind captured: {frac_captured}")
print()
orb_ang = vel / a
print(f"Orbital angular velocity: {orb_ang}")
print()
lamb= 1.0
rcirc_to_a = (m1**3 * (m1 + m2) / (16 * lamb**4 * m2**4)) * (r2 / a)**4
print(f"Circularization radius fraction compared to orbital sep: {rcirc_to_a}")
print(f"Circularization radius in m: {rcirc_to_a*a:8.8}")
print(f"Circularization radius in Rg: {rcirc_to_a * a / rg:8.8}") 

Orbital velocity: 325789.53939213336 m / s

Fraction of wind captured: 0.006408145820594852

Orbital angular velocity: 6.117755735901246e-06 1 / s

Circularization radius fraction compared to orbital sep: 7.987309258119215e-06
Circularization radius in m: 425349.08 m
Circularization radius in Rg: 13.335875


# For Cyg X-1 (M1 = Black Hole, M2 = Companion).

Orosz, J. A., McClintock, J. E., Aufdenberg, J. P., Remillard, R. A., Reid, M. J., Narayan, R., & Gou, L. (2011). THE MASS OF THE BLACK HOLE IN CYGNUS X-1. Astrophysical Journal, 742(2), 84. http://doi.org/10.1088/0004-637X/742/2/84

In [ ]:
m1 = 14.8 * M_sun
m2 = 19.2 * M_sun
period = 5.5999 * u.day

# Orbital Parameters

In [ ]:
period = period.to(u.s)
a3 = G * (period**2) * ( (m1 + m2)/ m1*m2) / (4*pi)
a = a3**(1/3.)
print 'Separation (m): ', a

a.to(u.au)
print 'Separation (AU): ', a.to(u.au)
print 'Light seconds: ', a / c

In [ ]:
rg = (2 * m1 * G) / (c**2)
print 'Schwarzschild Radius: ', rg

In [ ]:
# Flux in flare region
flux = 4.172e-11 / (u.cm)**2
d = (50 * u.kpc).to(u.cm)

area = 4 * np.pi *d**2
print(area * flux)

In [2]:
print(5 * np.sin(38*u.deg))

3.0783073766282913
